In [ ]:
import pickle
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from console1 import generate_partition
from console1 import NewHypercube
from console1 import make_cube_list
from console1 import single_subset_norm
from console1 import two_subset_norm

# F_P = {
#     0: [0,1],
#     1: [2,3],
#     2: [4],
#     3: [5],
#     4: [6,7,8,9,10],
#     5: [11],
#     6: [12,13],
# }

# filepath = 'C:/Users/User/Desktop/folder/pyfile/shap_residual/savedata/combined_CH.pkl'
# with open(filepath, 'rb') as f:
#     vertex_df = pickle.load(f)

# print(vertex_df.shape)
# print(vertex_df.iloc[4,6])
# vertex_df.iloc[4,6] = 1e-6
# print(vertex_df.iloc[4,6])
# datalist = make_cube_list(vertex_df, F_P)
# pgd_df, r_df, scn_df = single_subset_norm(datalist, F_P)
# pgd_df2, r_df2, scn_df2 = two_subset_norm(datalist, F_P)


In [ ]:
# a,b,c = generate_partition(F_P)
# data = {'null': [0],
#  '0': [1],
#  '1': [0],
#  '2': [0],
#  '0, 1': [1],
#  '0, 2': [1],
#  '1, 2': [2],
#  '0, 1, 2': [3],
#  }
# import pandas as pd
# data = pd.DataFrame(data)
# data.columns = [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]
# vertex_df = data
# F_P = {
#     0: [0],
#     1: [1],
#     2: [2],
# }
# from console1 import single_subset_norm
# from console1 import make_cube_list
# datalist = make_cube_list(vertex_df, F_P)
# pgd_df, r_df, scn_df = single_subset_norm(datalist, F_P)
# residual_0 = [ 4.44089210e-16,  1.33226763e-15,  8.88178420e-16,  2.22044605e-16,
#   2.22044605e-16 ,-6.66133815e-16 ,-2.22044605e-16 ,-2.22044605e-16,
#   2.22044605e-16  ,6.66133815e-16  ,6.66133815e-16  ,2.22044605e-16]
# residual_1= [ 4.4408921e-16,  5.0000000e-01, -5.0000000e-01,  5.0000000e-01,
#  -5.0000000e-01 , 0.0000000e+00 , 5.0000000e-01 , 8.8817842e-16,
#  -5.0000000e-01  ,5.0000000e-01 ,-5.0000000e-01, -4.4408921e-16]
# residual_2 = [ 0.0000000e+00, -5.0000000e-01,  5.0000000e-01, -5.0000000e-01,
#   5.0000000e-01 ,-4.4408921e-16 ,-5.0000000e-01 , 8.8817842e-16,
#   5.0000000e-01 ,-5.0000000e-01  ,5.0000000e-01 ,-4.4408921e-16]
# import numpy as np
# r1, r2 ,r3 = np.array(residual_0), np.array(residual_1),  np.array(residual_2)
# np.set_printoptions(precision=6, suppress=True)
# print(r1)
# print(r2)
# print(r3)
# print(r1+r2)
# print(r1+r3)
# print(r2+r3)
# print(np.linalg.norm(r2+r3))

residual_m[0][0]
[ 4.44089210e-16  1.33226763e-15  8.88178420e-16  2.22044605e-16
  2.22044605e-16 -6.66133815e-16 -2.22044605e-16 -2.22044605e-16
  2.22044605e-16  6.66133815e-16  6.66133815e-16  2.22044605e-16]
residual_m[0][1]
[ 4.4408921e-16  5.0000000e-01 -5.0000000e-01  5.0000000e-01
 -5.0000000e-01  0.0000000e+00  5.0000000e-01  8.8817842e-16
 -5.0000000e-01  5.0000000e-01 -5.0000000e-01 -4.4408921e-16]
residual_m[0][2]
[ 0.0000000e+00 -5.0000000e-01  5.0000000e-01 -5.0000000e-01
  5.0000000e-01 -4.4408921e-16 -5.0000000e-01  8.8817842e-16
  5.0000000e-01 -5.0000000e-01  5.0000000e-01 -4.4408921e-16]
0    1.047382e-15
1    5.000000e-01
2    5.000000e-01
dtype: float64


[ 0.  0.  0.  0.  0. -0. -0. -0.  0.  0.  0.  0.]
[ 0.   0.5 -0.5  0.5 -0.5  0.   0.5  0.  -0.5  0.5 -0.5 -0. ]
[ 0.  -0.5  0.5 -0.5  0.5 -0.  -0.5  0.   0.5 -0.5  0.5 -0. ]
[ 0.   0.5 -0.5  0.5 -0.5 -0.   0.5  0.  -0.5  0.5 -0.5 -0. ]
[ 0.  -0.5  0.5 -0.5  0.5 -0.  -0.5  0.   0.5 -0.5  0.5 -0. ]
[ 0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.]
2.0829630293515367e-15
